In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
main_data_df = pd.read_csv('main_data_from_2013.csv', index_col=0)

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 13.9 s


In [3]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x + '-0'
    else:
        return x

def create_land_specs_df():
    df = pd.read_csv('./토지특성정보/AL_11_D195_20200925.csv', encoding='euc-kr', usecols=[2, 4, 5, 7, 8, 10, 11, 13, 17, 19, 21, 23])
    
    #house_list = ['단독, 다세대', '주거기타', '아파트', '연립']
    
    df = df[df['대장구분명'] == '일반']#.reset_index(drop=True)
    #df = df[df['토지이동상황'].isin(house_list)]
    df = df[df['기준월'] == 1].reset_index(drop=True)
    
    df.fillna('지정되지않음', inplace=True)
    
    df['전체주소'] = df['법정동명'] + ' ' + df['지번'].apply(landnum_modifier)
    
    categorical_cols = ['지목명', '용도지역명1', '지형높이', '지형형상', '도로접면']
    
    for col in tqdm(categorical_cols, position=0):
        unique_values = df[col].unique()
        for v in unique_values:
            df[col + '_' + '{}'.format(v)] = 0
            df[col + '_' + '{}'.format(v)] = df[col].apply(lambda x: 1 if x == v else 0)
    
    df.drop(columns=categorical_cols + ['법정동명', '지번', '토지이동상황', '기준월', '대장구분명'], inplace=True)
    
    return df

In [4]:
#land_specs_df = create_land_specs_df()

In [5]:
#land_specs_df.to_csv('simple_land_specs.csv')

In [6]:
%%time
land_specs_df = pd.read_csv('simple_land_specs.csv', index_col=0)

Wall time: 45.8 s


In [7]:
'''land_prices_df = pd.read_csv('land_prices_df_20201224-130609.csv')
land_prices_df = land_prices_df[land_prices_df['년'] >= 2013].reset_index(drop=True)
land_prices_df.to_csv('land_prices_from_2013.csv')'''

"land_prices_df = pd.read_csv('land_prices_df_20201224-130609.csv')\nland_prices_df = land_prices_df[land_prices_df['년'] >= 2013].reset_index(drop=True)\nland_prices_df.to_csv('land_prices_from_2013.csv')"

In [8]:
%%time
land_prices_df = pd.read_csv('land_prices_from_2013.csv', index_col=0)

Wall time: 5.1 s


In [21]:
def read_coor_data1():
    basedir = './좌표데이터/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi').drop_duplicates()
                            
            df['도로명주소'] = df[3] + ' ' + df[4] + ' ' + df[7] + ' ' + df[9].astype('str') + '-' + df[10].astype('str')
            
            df['x좌표'] = df[16]
            df['y좌표'] = df[17]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

def read_coor_data2():
    basedir = './좌표데이터2/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None, usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24]).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi', usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24])\
                .drop_duplicates()
                
            df['도로명주소'] = df[1] + ' ' + df[2] + ' ' + df[5] + ' ' + df[7].astype('str') + '-' + df[8].astype('str')
            
            df['x좌표'] = df[23]
            df['y좌표'] = df[24]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

def read_coor_data3():
    basedir = './좌표데이터3/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            df = pd.read_csv(basedir + filename, index_col=0).drop_duplicates()            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['전체주소'], keep='last').reset_index(drop=True)
    concat_df.columns = ['지번주소', 'x좌표', 'y좌표']
    
    return concat_df

def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x
    
def read_coor_data():
    roadname_coor_df1 = read_coor_data1()
    roadname_coor_df2 = read_coor_data2()
    
    roadname_coor_df = pd.concat([roadname_coor_df1, roadname_coor_df2]).drop_duplicates(subset=['도로명주소'], keep='last')\
    .reset_index(drop=True)
    
    landnum_coor_df = read_coor_data3()
        
    return roadname_coor_df, landnum_coor_df

In [22]:
roadname_coor_df, landnum_coor_df = read_coor_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 35.85it/s]


# Merging Datasets

In [9]:
print(main_data_df.shape)
print(land_specs_df.shape)
print(land_prices_df.shape)

(3378284, 21)
(7289090, 71)
(7436007, 3)


In [10]:
display(main_data_df.head())
display(land_specs_df.head())
display(land_prices_df.head())

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0


,기준년도,토지면적,전체주소,지목명_대,지목명_도로,지목명_하천,지목명_임야,지목명_공원,지목명_구거,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_제방,지목명_수도용지,지목명_전,지목명_묘지,지목명_체육용지,지목명_공장용지,지목명_창고용지,지목명_유지,지목명_답,지목명_유원지,지목명_과수원,지목명_목장용지,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_보전녹지지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각
0,2013,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2014,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,2016,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2017,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


,전체주소,년,공시지가(만원)
0,서울특별시 종로구 청운동 1-0,2013,284.0
1,서울특별시 종로구 청운동 1-0,2014,293.3
2,서울특별시 종로구 청운동 1-0,2015,305.3
3,서울특별시 종로구 청운동 1-0,2016,323.4
4,서울특별시 종로구 청운동 1-0,2017,348.8


In [12]:
main_data_df.rename(columns={'year':'년', 'month':'월'}, inplace=True)

In [14]:
land_prices_df.rename(columns={'전체주소':'지번주소'}, inplace=True)

In [15]:
prac_main_df = main_data_df.merge(land_prices_df, on=['지번주소', '년'])
prac_main_df.shape

(3286171, 22)

In [16]:
prac_main_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,년,월,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원)
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0


In [17]:
land_specs_df.rename(columns={'전체주소':'지번주소', '기준년도':'년'}, inplace=True)

In [18]:
prac_merged_df = prac_main_df.merge(land_specs_df, on=['지번주소', '년'])
prac_merged_df.shape

(3260586, 91)

In [19]:
prac_merged_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,년,월,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_하천,지목명_임야,지목명_공원,지목명_구거,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_제방,지목명_수도용지,지목명_전,지목명_묘지,지목명_체육용지,지목명_공장용지,지목명_창고용지,지목명_유지,지목명_답,지목명_유원지,지목명_과수원,지목명_목장용지,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_보전녹지지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [20]:
prac_merged_df.isna().sum()

시군구                     0
번지                      0
본번                      0
부번                      0
건물명                     7
전용면적(㎡)                 0
계약년월                    0
계약일                     0
가격(만원)                  0
층                       0
건축년도                    0
도로명                 22329
가격/면적                   0
년                       0
월                       0
시                       0
구                       0
전월세매매구분                 0
건물종류                    0
지번주소                    0
도로명주소               22329
공시지가(만원)                0
토지면적                    0
지목명_대                   0
지목명_도로                  0
지목명_하천                  0
지목명_임야                  0
지목명_공원                  0
지목명_구거                  0
지목명_학교용지                0
지목명_종교용지                0
지목명_사적지                 0
지목명_주차장                 0
지목명_철도용지                0
지목명_주유소용지               0
지목명_잡종지                 0
지목명_제방                  0
지목명_수도용지                0
지목명_전       

In [26]:
prac_merged_df.dropna(inplace=True)
prac_merged_df.shape

(3238250, 91)

In [28]:
prac_merged_df.reset_index(drop=True, inplace=True)

In [29]:
prac_merged_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,년,월,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_하천,지목명_임야,지목명_공원,지목명_구거,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_제방,지목명_수도용지,지목명_전,지목명_묘지,지목명_체육용지,지목명_공장용지,지목명_창고용지,지목명_유지,지목명_답,지목명_유원지,지목명_과수원,지목명_목장용지,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_보전녹지지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [43]:
description = prac_merged_df.describe()

In [45]:
cols_to_drop = []
for col in description.columns:
    if description[col]['std'] == 0:
        cols_to_drop.append(col)

In [46]:
len(cols_to_drop)

12

In [47]:
cols_to_drop

['지목명_하천',
 '지목명_구거',
 '지목명_사적지',
 '지목명_제방',
 '지목명_묘지',
 '지목명_체육용지',
 '지목명_유지',
 '지목명_유원지',
 '지목명_과수원',
 '지목명_목장용지',
 '용도지역명1_보전녹지지역',
 '용도지역명1_생산녹지지역']

In [48]:
prac_merged_df.drop(columns=cols_to_drop, inplace=True)

In [49]:
prac_merged_df.shape

(3238250, 79)

In [50]:
print(roadname_coor_df.shape)
roadname_coor_df.head()

(566869, 3)


,도로명주소,x좌표,y좌표
0,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06


In [51]:
print(landnum_coor_df.shape)
landnum_coor_df.head()

(90867, 3)


,지번주소,x좌표,y좌표
0,서울특별시 서초구 고무래로,956676.746259,1.945089e+06
1,서울특별시 서초구 방배로 21-0,955386.073490,1.942583e+06
2,서울특별시 성동구 난계로 9-0,957972.833890,1.951036e+06
3,서울특별시 양천구 남부순환로85길 10-0,940382.983086,1.948322e+06
4,서울특별시 은평구 응암로16길 37-0,948796.942106,1.954765e+06


In [52]:
prac_merged_coor_df = prac_merged_df.merge(roadname_coor_df, on=['도로명주소'], how='left')
prac_merged_coor_df.shape

(3238250, 81)

In [53]:
prac_merged_coor_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,년,월,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06


In [58]:
prac_merged_coor_df = prac_merged_coor_df.merge(landnum_coor_df, on=['지번주소'], how='left')
prac_merged_coor_df.shape

(3238250, 83)

In [59]:
prac_merged_coor_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,년,월,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표_x,y좌표_x,x좌표_y,y좌표_y
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,960471.903756,1.942550e+06
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,960471.903756,1.942550e+06
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,960471.903756,1.942550e+06
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,960471.903756,1.942550e+06
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,960471.903756,1.942550e+06


In [60]:
prac_merged_coor_df.isna().sum()

시군구                      0
번지                       0
본번                       0
부번                       0
건물명                      0
전용면적(㎡)                  0
계약년월                     0
계약일                      0
가격(만원)                   0
층                        0
건축년도                     0
도로명                      0
가격/면적                    0
년                        0
월                        0
시                        0
구                        0
전월세매매구분                  0
건물종류                     0
지번주소                     0
도로명주소                    0
공시지가(만원)                 0
토지면적                     0
지목명_대                    0
지목명_도로                   0
지목명_임야                   0
지목명_공원                   0
지목명_학교용지                 0
지목명_종교용지                 0
지목명_주차장                  0
지목명_철도용지                 0
지목명_주유소용지                0
지목명_잡종지                  0
지목명_수도용지                 0
지목명_전                    0
지목명_공장용지                 0
지목명_창고용지                 0
지

In [76]:
def remove_both_nan_and_merge_coor_cols(df):
    gc.collect()
    df = df.copy()
    
    both_nan_df = df[(df['x좌표_x'].isna()) & (df['x좌표_y'].isna())]
    
    df = df.drop(both_nan_df.index).reset_index(drop=True)
    
    df['x좌표_x'].fillna(df['x좌표_y'], inplace=True)
    df['y좌표_x'].fillna(df['y좌표_y'], inplace=True)
    
    df.drop(columns=['x좌표_y', 'y좌표_y'], inplace=True)
    df.rename(columns={'x좌표_x':'x좌표', 'y좌표_x':'y좌표'}, inplace=True)
    
    return df

In [78]:
prac_merged_coor_df = remove_both_nan_and_merge_coor_cols(prac_merged_coor_df)
prac_merged_coor_df.shape

(3225086, 81)

In [79]:
prac_merged_coor_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,년,월,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06


In [82]:
cols_to_drop = ['시군구', '번지', '본번', '부번', '건물명', '계약년월', '도로명', '가격/면적', '시', '구', '지번주소', '도로명주소', '계약일']
prac_merged_coor_df.drop(columns=cols_to_drop, inplace=True)
prac_merged_coor_df.shape

(3225086, 69)

In [87]:
prac_merged_coor_df.head()

,전용면적(㎡),가격(만원),층,건축년도,년,월,전월세매매구분,건물종류,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표
0,77.75,57000,2,1988,2013,9,매매,아파트,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
1,77.75,57000,2,1988,2013,12,매매,아파트,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
2,77.75,35000,4,1988,2013,1,전세,아파트,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
3,77.75,32000,9,1988,2013,3,전세,아파트,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06
4,77.75,37000,1,1988,2013,9,전세,아파트,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06


In [4]:
def categorical_cols_to_binary(df):
    gc.collect()
    df = df.copy()
    
    categorical_cols = ['전월세매매구분', '건물종류']
    
    for col in tqdm(categorical_cols, position=0):
        unique_values = df[col].unique()
        for v in unique_values:
            df[col + '_' + '{}'.format(v)] = 0
            df[col + '_' + '{}'.format(v)] = df[col].apply(lambda x: 1 if x == v else 0)
    
    df.drop(columns=categorical_cols, inplace=True)
    
    return df

In [5]:
prac_df = categorical_cols_to_binary(prac_merged_coor_df)
prac_df.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.75s/it]


(3225086, 72)

In [ ]:
#prac_merged_coor_df.to_csv('prac_merged_coor_df.csv', index=False)

In [6]:
prac_df.head()

,전용면적(㎡),가격(만원),층,건축년도,년,월,공시지가(만원),토지면적,지목명_대,지목명_도로,지목명_임야,지목명_공원,지목명_학교용지,지목명_종교용지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_수도용지,지목명_전,지목명_공장용지,지목명_창고용지,지목명_답,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각,x좌표,y좌표,전월세매매구분_매매,전월세매매구분_전세,전월세매매구분_월세,건물종류_아파트,건물종류_오피스텔,건물종류_연립다세대
0,77.75,57000,2,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0
1,77.75,57000,2,1988,2013,12,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,1,0,0,1,0,0
2,77.75,35000,4,1988,2013,1,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0
3,77.75,32000,9,1988,2013,3,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0
4,77.75,37000,1,1988,2013,9,921.0,1168.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,960471.903756,1.942550e+06,0,1,0,1,0,0


In [9]:
prac_df.to_csv('main_data_{}.csv'.format(pd.datetime.today().strftime('%y%m%d-%H%M%S')), index=False)

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.
